**Importing Libraries and Data**

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
from mySSA import mySSA
import numpy as np
import warnings
import gc
import os
import sys

#Ignoring warnings
warnings.filterwarnings("ignore")

#Reading in the data and extracting unique account values
BoostedTransactions = pd.read_csv("Longer Subset of Transaction Data (14 Months).csv")
AccountIDs = BoostedTransactions["AccountId"].values
UniqueAccounts = set(AccountIDs)

#Converting transaction dates into months
BoostedTransactions["TransactionDate"] = pd.to_datetime(BoostedTransactions["TransactionDate"])
BoostedTransactions["TransactionDate"] = BoostedTransactions["TransactionDate"].dt.to_period("M").dt.to_timestamp()

#Grouping transactions by account and month and summating each grouping's net transaction amount
GroupedSumOfTransactionsByAccountAndMonth = BoostedTransactions.groupby(by=["AccountId", "TransactionDate"]).agg(["sum"])["Amount"]
print(GroupedSumOfTransactionsByAccountAndMonth)

#Creating a list to store each account's time series in
DataFrames = []

#Imputing 0 values into the gaps in each account's time series
for i in range(0, len(UniqueAccounts)):
  ResetDataFrame = GroupedSumOfTransactionsByAccountAndMonth.reset_index()
  TimeSeries = ResetDataFrame[ResetDataFrame["AccountId"] == list(UniqueAccounts)[i]]
  TimeSeries = TimeSeries.set_index("TransactionDate")

  TimeSeries = TimeSeries.resample("MS").sum()
  for j in range(0, len(TimeSeries)):
    if TimeSeries.iloc[j]["AccountId"] == 0:
      TimeSeries.at[TimeSeries.index[j], "AccountId"] = list(UniqueAccounts)[i]

  DataFrames.append(TimeSeries)

#Combining all accounts' time series into a new grouped dataset
NewGroupedDataset = pd.concat(DataFrames)
NewGroupedDataset = NewGroupedDataset.reset_index()
NewGroupedDataset = NewGroupedDataset.set_index(["AccountId", "TransactionDate"])
print(NewGroupedDataset)

#Calculating the cumulative transaction sum over the time period for each account
GroupedCumulativeSumOfTransactionsByAccountAndMonth = NewGroupedDataset.groupby(level=0).cumsum().reset_index()
print(GroupedCumulativeSumOfTransactionsByAccountAndMonth)

                                                          sum
AccountId                            TransactionDate         
0003a5ae-0c77-4372-b44d-882ef9874a28 2019-05-01      -5541.31
                                     2019-06-01      -3601.46
                                     2019-07-01       3151.74
                                     2019-08-01      -4801.13
                                     2019-09-01       3890.05
...                                                       ...
fff7f00c-c869-4310-b705-4503538f5ecf 2020-03-01       2974.84
                                     2020-04-01        382.93
                                     2020-05-01        166.15
                                     2020-06-01       -550.89
                                     2020-07-01      -2721.18

[7632 rows x 1 columns]
                                                            sum
AccountId                            TransactionDate           
2846c3f5-08a2-4683-b9e5-cf800a931ddc 2019

**Key SSA Loop**

This will only execute if the mySSA.py Python class is imported into the Colab environment.

In [ ]:
#Creating models and forecasting for all accounts
for i in UniqueAccounts:
    print(i)
    try:
        #Selecting only the current account's time series
        ForecastingTestTimeSeries = GroupedCumulativeSumOfTransactionsByAccountAndMonth[GroupedCumulativeSumOfTransactionsByAccountAndMonth["AccountId"] == i][["TransactionDate", "sum"]]

        ForecastingTestTimeSeries = ForecastingTestTimeSeries.set_index("TransactionDate")

        #Calculating the cutoff point between training and testing sets
        TrainEnd = int(0.7 * len(ForecastingTestTimeSeries))

        #Splitting the dataset into training and testing sets
        TrainData = ForecastingTestTimeSeries[:TrainEnd]
        TestData = ForecastingTestTimeSeries[TrainEnd:]

        #Setting the window length L to N/2 unless N/2 is less than 2
        if len(TrainData) <= 2:
            L = len(TrainData)
        else:
            L = max(2, len(TrainData) // 2)

        #Calculating K based on L and selecting the suspected frequency based on significant autocorrelations
        #It is set as 0 if none are detected
        K = len(TrainData) - L + 1
        try:
            if K <= 1:
              MostSignificantLag = 0
            else:
              ACFValues = acf(TrainData.iloc[:, 0], nlags=K, fft=False)
              MostSignificantLag = ACFValues[1:].argmax() + 1
              Threshold = 0.2
              if np.isnan(ACFValues).any():
                  MostSignificantLag = 0
              else:
                  MostSignificantLag = ACFValues[1:].argmax() + 1
                  Threshold = 0.2
                  if ACFValues[MostSignificantLag] < Threshold:
                      MostSignificantLag = 0
        except Exception as e:
            MostSignificantLag = 0

        #Training an SSA model, embedding and decomposing
        if TrainData.empty or TrainData.isnull().all().any():
            print(f"Skipping account {i} due to empty or invalid TrainData")
            #print(TrainData)
            break
        SSA = mySSA(TrainData)

        SSA.embed(embedding_dimension=K, suspected_frequency=MostSignificantLag)

        SSA.decompose()

        #Analysing signal contributions
        Contributions = SSA.view_s_contributions(adjust_scale=True, return_df=True)

        #Creating lists to store stream numbers and error statistics
        StreamNumbers = []
        ErrorsRMSE = []
        ErrorsSI = []

        #Trying all potential stream numbers to see which one returns the lowest RMSE
        for h in range(1, (len(Contributions)+1)):
            Streams = [j for j in range(h)]
            if hasattr(SSA, 'X_com_hat'):
                del SSA.X_com_hat
            Forecast = SSA.forecast_recurrent(steps_ahead=(len(ForecastingTestTimeSeries)-TrainEnd), singular_values=Streams, return_df=True)
            Forecast["Forecast"] = Forecast["Forecast"].clip(lower=-1e6, upper=1e6)
            Forecast["Forecast"] = Forecast["Forecast"].fillna(0)
            try:
                RMSE = root_mean_squared_error(TestData["sum"], Forecast["Forecast"][TrainEnd:])
                StreamNumbers.append(h)
                ErrorsRMSE.append(RMSE)
                ErrorsSI.append(abs(RMSE/abs(TestData["sum"]).mean()))
            except ValueError as eV:
                print("Bad RMSE value encountered.")
                print(TestData["sum"])
                print(Forecast["Forecast"])
                print(RMSE)
                print(abs(RMSE/abs(TestData["sum"]).mean()))
                ErrorType, ErrorObject, ErrorTraceback = sys.exc_info()

                ErrorFilename = os.path.split(
                    ErrorTraceback.tb_frame.f_code.co_filename
                )[1]

                ErrorMessage = str(eV)

                ErrorLineNumber = ErrorTraceback.tb_lineno

                print(f'Exception Type: {ErrorType}')

                print(f'Exception Filename: {ErrorFilename}')

                print(f'Exception Line Number: {ErrorLineNumber}')

                print(f'Exception Message: {ErrorMessage}')
                break

        ErrorData = pd.DataFrame({"Streams" : StreamNumbers,
                         "RMSE" : ErrorsRMSE,
                         "SI" : ErrorsSI})
        #Taking the stream number with the lowest RMSE and appending its error statistics into the errors CSV
        try:
          IdealNumberOfStreams = ErrorData.loc[ErrorData["RMSE"].idxmin()]["Streams"]
          #print(IdealNumberOfStreams)
          LowestRMSE = ErrorData.loc[ErrorData["RMSE"].idxmin()]["RMSE"]
          LowestSI = ErrorData.loc[ErrorData["SI"].idxmin()]["SI"]
          AccountError = pd.DataFrame({"AccountID" : [i],
                                        "RMSE" : [LowestRMSE],
                                        "SI" : [LowestSI]})
          AccountError.to_csv("SSA Longer Subset Errors (Cumulative).csv", mode='a', header=not os.path.exists("SSA Longer Subset Errors (Cumulative).csv"), index=False)
        except ValueError:
          print("ValueError encountered")
          print(ErrorData)
          break

        #Forecasting the next 12 months using the chosen forecast
        IdealStreams = [k for k in range(int(IdealNumberOfStreams))]

        if hasattr(SSA, 'X_com_hat'):
            del SSA.X_com_hat

        BestForecast = SSA.forecast_recurrent(steps_ahead=((len(ForecastingTestTimeSeries)-TrainEnd)+12), singular_values=IdealStreams, return_df=True)

        FutureForecast = BestForecast[-12:]["Forecast"].values

        #Calculating starting amount, ending amount and net change and appending them into the forecasts CSV file
        NetChange = FutureForecast[-1] - FutureForecast[0]
        StartingAmount = FutureForecast[0]
        EndingAmount = FutureForecast[-1]

        ForecastData = pd.DataFrame({"AccountID" : [i],
                                     "Net Change over Forecast Period" : [NetChange],
                                     "Starting Amount" : [StartingAmount],
                                     "Ending Amount" : [EndingAmount]})

        ForecastData.to_csv("SSA Longer Subset Forecasts (Cumulative).csv", mode='a', header=not os.path.exists("SSA Longer Subset Forecasts (Cumulative).csv"), index=False)

        print("Processed account number",i)
    #Where an error is returned, the user is informed about it
    except Exception as e:
        print(f"Error encountered processing account number {i}.")
        ErrorType, ErrorObject, ErrorTraceback = sys.exc_info()

        ErrorFilename = os.path.split(
            ErrorTraceback.tb_frame.f_code.co_filename
        )[1]

        ErrorMessage = str(e)

        ErrorLineNumber = ErrorTraceback.tb_lineno

        print(f'Exception Type: {ErrorType}')

        print(f'Exception Filename: {ErrorFilename}')

        print(f'Exception Line Number: {ErrorLineNumber}')

        print(f'Exception Message: {ErrorMessage}')
        break
    #Deleting model data to conserve RAM
    finally:
        del SSA, Forecast, TrainData, TestData, ForecastingTestTimeSeries
        gc.collect()

print("Code executed successfully.")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#Calculating averaged error statistics
Errors = pd.read_csv("SSA Errors.csv")
MeanRMSE = Errors["RMSE"].mean()
print("Mean RMSE:",MeanRMSE)
MedianRMSE = Errors["RMSE"].median()
print("Median RMSE:",MedianRMSE)
MeanSI = Errors["SI"].mean()
print("Mean SI:",MeanSI)
MedianSI = Errors["SI"].median()
print("Median SI:",MedianSI)

Mean RMSE: 525789.3850472847
Median RMSE: 23368.34271150353
Mean SI: 0.7174521918825119
Median SI: 0.5657585588450607
